In [1]:
%load_ext autoreload
%autoreload 2

from datasets import Zone1
ds = Zone1('data/zone1_prior_load.parquet')


In [2]:
df = ds.data.copy()
import pandas as pd
import numpy as np
day = 24*60*60
year = (365.2425)*day
timestamp_s = df.index.to_series().map(pd.Timestamp.timestamp)
df['Daily sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Daily cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Weekly sin'] = np.sin(timestamp_s * (2 * np.pi / (7*day)))
df['Weekly cos'] = np.cos(timestamp_s * (2 * np.pi / (7*day)))
df['Yearly sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Yearly cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [3]:
from validation import Error, walkforward
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

In [4]:
knn = KNeighborsRegressor(7, weights='distance')
(predictions, errors) = walkforward(knn, df, ds.data[ds.actual],
                                    ds.validation_start, ds.validation_end, Zone1.next_hour,
                                    ds.correlated_columns +  ['Daily sin', 'Daily cos', 'Weekly sin', 'Weekly cos', 'Yearly sin', 'Yearly cos'])
overall_error = Error(ds.validation_data[ds.actual], np.concatenate(predictions))
print(overall_error)

Error(mae=516.4689863273425, max=3416.3899998856414, total=4524268.320227515)


In [16]:
mtlf_error = Error(ds.validation_data[ds.actual], ds.validation_data[ds.mtlf])
print(f'MTLF {mtlf_error}')

MTLF Error(mae=222.98570091324197, max=2217.33, total=1953354.7399999993)
